In [278]:
import random
import pandas as pd
import numpy as np
import os
import cv2

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2
import torchvision.models as models

from tqdm.auto import tqdm
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.model_selection import StratifiedKFold

import warnings
warnings.filterwarnings(action='ignore')

In [279]:
device = torch.device("mps") if torch.backends.mps.is_available() else "cpu"
print(f"device : {device}")

device : mps


In [280]:
CFG = {
    'VIDEO_LENGTH':50,
    'IMG_SIZE':128,
    'EPOCHS':10,
    'LEARNING_RATE':3e-4,
    'BATCH_SIZE':4,
    'SEED':42
}

In [281]:
train_df = pd.read_csv('./input/train.csv')
train_df.head()

,sample_id,video_path,label
0,TRAIN_0000,./train/TRAIN_0000.mp4,7
1,TRAIN_0001,./train/TRAIN_0001.mp4,7
2,TRAIN_0002,./train/TRAIN_0002.mp4,0
3,TRAIN_0003,./train/TRAIN_0003.mp4,0
4,TRAIN_0004,./train/TRAIN_0004.mp4,1


Crash + Ego-involved Class
0: Non-crash
1: ego involved
2: Non ego involved

In [282]:
# Non-crash
df0 = train_df[train_df['label'] == 0]

# ego involved
df1 = train_df[train_df['label'] <= 6]
df1 = df1[df1['label'] >= 1]
df1['label'] = 1

# Non ego involved
df2 = train_df[train_df['label'] >= 7]
df2['label'] = 2

In [283]:
crash_ego_df = pd.concat([df0, df1, df2])

In [284]:
crash_ego_df = crash_ego_df.sort_index()
crash_ego_df

,sample_id,video_path,label
0,TRAIN_0000,./train/TRAIN_0000.mp4,2
1,TRAIN_0001,./train/TRAIN_0001.mp4,2
2,TRAIN_0002,./train/TRAIN_0002.mp4,0
3,TRAIN_0003,./train/TRAIN_0003.mp4,0
4,TRAIN_0004,./train/TRAIN_0004.mp4,1
...,...,...,...
2693,TRAIN_2693,./train/TRAIN_2693.mp4,1
2694,TRAIN_2694,./train/TRAIN_2694.mp4,1
2695,TRAIN_2695,./train/TRAIN_2695.mp4,0
2696,TRAIN_2696,./train/TRAIN_2696.mp4,0


In [285]:
drop_non_crash = train_df[train_df['label'] >= 1]
drop_non_crash

,sample_id,video_path,label
0,TRAIN_0000,./train/TRAIN_0000.mp4,7
1,TRAIN_0001,./train/TRAIN_0001.mp4,7
4,TRAIN_0004,./train/TRAIN_0004.mp4,1
6,TRAIN_0006,./train/TRAIN_0006.mp4,3
7,TRAIN_0007,./train/TRAIN_0007.mp4,7
...,...,...,...
2685,TRAIN_2685,./train/TRAIN_2685.mp4,8
2689,TRAIN_2689,./train/TRAIN_2689.mp4,1
2692,TRAIN_2692,./train/TRAIN_2692.mp4,7
2693,TRAIN_2693,./train/TRAIN_2693.mp4,3


Timing Class
0: Night
1: Day

In [286]:
# Night
df0 = drop_non_crash[drop_non_crash['label'] % 2 == 0]
df0['label'] = 0

# Day
df1 = drop_non_crash[drop_non_crash['label'] % 2 == 1]
df1['label'] = 1

In [287]:
timing_df = pd.concat([df0, df1])
timing_df = timing_df.sort_index()
timing_df

,sample_id,video_path,label
0,TRAIN_0000,./train/TRAIN_0000.mp4,1
1,TRAIN_0001,./train/TRAIN_0001.mp4,1
4,TRAIN_0004,./train/TRAIN_0004.mp4,1
6,TRAIN_0006,./train/TRAIN_0006.mp4,1
7,TRAIN_0007,./train/TRAIN_0007.mp4,1
...,...,...,...
2685,TRAIN_2685,./train/TRAIN_2685.mp4,0
2689,TRAIN_2689,./train/TRAIN_2689.mp4,1
2692,TRAIN_2692,./train/TRAIN_2692.mp4,1
2693,TRAIN_2693,./train/TRAIN_2693.mp4,1


Weather Class
0: Normal
1: Snowy
2: Rainy

In [288]:
def define_weather(x):
    if 1<=x<=2 or 7<=x<=8:
        return 0
    elif 3<=x<=4 or 9<=x<=10:
        return 1
    else:
        return 2

In [289]:
weather_df = pd.DataFrame(drop_non_crash)
weather_df['label'] = drop_non_crash['label'].apply(define_weather)
weather_df

,sample_id,video_path,label
0,TRAIN_0000,./train/TRAIN_0000.mp4,0
1,TRAIN_0001,./train/TRAIN_0001.mp4,0
4,TRAIN_0004,./train/TRAIN_0004.mp4,0
6,TRAIN_0006,./train/TRAIN_0006.mp4,1
7,TRAIN_0007,./train/TRAIN_0007.mp4,0
...,...,...,...
2685,TRAIN_2685,./train/TRAIN_2685.mp4,0
2689,TRAIN_2689,./train/TRAIN_2689.mp4,0
2692,TRAIN_2692,./train/TRAIN_2692.mp4,0
2693,TRAIN_2693,./train/TRAIN_2693.mp4,1
